In [ ]:
from gogettr import PublicClient
client = PublicClient()
import datetime
import pandas as pd
import re
import sqlite3
# Creating a SQL connection to our SQLite database
con = sqlite3.connect("gettr.sqlite")
#get lastest date
df= pd.read_sql('select * from gettr_table', con)
most_recent_date = df['date'].max()

# List contains a total of 77 verified "fringe" users on Gettr
list = ['indynewsmedia','rwmalonemd','joerogan','tpusa','timcast','theofficertatum','hodgetwins','thelaurenchen','rudygiuliani',
        'anncoulter','thequartering','gavinmcinnes','dineshdsouza','awakenwithjp','diamondandsilk',
        'infowars','blacklistednews','news_ntd','clayandbuckshow','tonybrunoshow','waynedupreeshow',
        'joetalkshow','gatewaypundit','monicamatthews','resisttm','nationalpulse','breaking911',
        'rsbnetwork','justthenews','healthnutnews','truthaboutcovid','thetruthgazette',
        'thedamanifelder','hotair','scottfantasy','project_veritas','dailynews',
        'lifenewshq','coronacommittee','rebelnewsonline','plandemicseries','drzevzelenko','malcolmoutloud',
        'defiantamerica','winningpatriot','stacyontheright','mediarightnews','leeroypress','twitchy',
        'dschlopesisback','villagenorth','richardursomd','ajeemontes','zerohedge','tspooky','rohll5',
        'carlossimancas','ivoryhecker','william5849','dloesch','ladyredwave','gonewiththerain','realstewpeters',
        'bigleaguepol','stevegruber','tweettruth2me','chiefnerd','davidjharrisjr','brandonstraka',
        'joeysalads','realmattcouch','serremmy','redvoicemedia','mikhailaaleksis','jasonmillerindc',
        'childrenshd','barbiewakeupusa']
# Function is pulling all the posts (no time range) from the list of users on Gettr
def test_user_posts():
    allposts = []
    for user in list:
        try: 
            posts = client.user_activity(username=user)
            allposts.extend(posts)
        except AttributeError:
            continue
    return allposts


output = test_user_posts()
df = pd.DataFrame.from_records(output)


#Creating function to adjust cdate structure
def converttimestamp(timestamp):
    timestamp = timestamp / 1000
    dt = datetime.datetime.fromtimestamp(timestamp)
    return dt.strftime('%m-%d-%Y')


#data cleaning
df = df[["cdate", "receiver_id", "txt"]].copy()
df.cdate = df.cdate.apply(converttimestamp)
df['cdate'] = pd.to_datetime(df['cdate'])
df['txt'] = df['txt'].fillna('')
df['txt'] = df['txt'].astype(str)
df = df.sort_values(by='cdate')
df = df.rename({
    'cdate': 'date',
    'receiver_id': 'username',
    'txt': 'post'
},
               axis=1)

#get posts from the lastest date
most_date = (df['date'] > most_recent_date)
df =df.loc[most_date]

#remove reposts
df = df[~df.post.str.contains("RT")]

#filter covid posts
df = df[df.post.str.contains(
    "covid|covid-19|covid19|sars-cov-2|coronavirus|chinavirus|china virus",
    flags=re.IGNORECASE)]

# Converting pandas dataframe into sql database
df.to_sql('gettr_table', con, index=False, if_exists='append')